<a href="https://colab.research.google.com/github/BChun11/DATA3001/blob/main/DATA3001_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Download all the files into google colab environment
!git clone https://github.com/nokuik/KDDI-IoT-2019.git

Cloning into 'KDDI-IoT-2019'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 42 (delta 1), reused 9 (delta 1), pack-reused 33
Receiving objects: 100% (42/42), 776.84 MiB | 21.78 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (31/31), done.


In [3]:
# List all '.tar.gz files in ipfix directory
%cd KDDI-IoT-2019
%cd ipfix
!ls *.tar.gz

/content/KDDI-IoT-2019
/content/KDDI-IoT-2019/ipfix
 amazon_echo_gen2.tar.gz		        nature_remo.tar.gz
 au_network_camera.tar.gz		        panasonic_doorphone.tar.gz
 au_wireless_adapter.tar.gz		        philips_hue_bridge.tar.gz
 bitfinder_awair_breathe_easy.tar.gz	       'planex_camera_one_shot!.tar.gz'
 candy_house_sesami_wi-fi_access_point.tar.gz   planex_smacam_outdoor.tar.gz
 irobot_roomba.tar.gz			        planex_smacam_pantilt.tar.gz
 jvc_kenwood_cu-hb1.tar.gz		        powerelectric_wi-fi_plug.tar.gz
 jvc_kenwood_hdtv_ip_camera.tar.gz	        qrio_hub.tar.gz
 line_clova_wave.tar.gz			        sony_network_camera.tar.gz
 link_japan_eremote.tar.gz		        sony_smart_speaker.tar.gz
 mouse_computer_room_hub.tar.gz		        xiaomi_mijia_led.tar.gz


In [4]:
# List all tar.gz files and store them in a variable
files = !ls -1 *.tar.gz

# Extract each tar.gz file
for file in files:
    print(f"Extract {file} ")
    !tar -xzvf {file}

Extract amazon_echo_gen2.tar.gz 
amazon_echo_gen2.json
Extract au_network_camera.tar.gz 
au_network_camera.json
Extract au_wireless_adapter.tar.gz 
au_wireless_adapter.json
Extract bitfinder_awair_breathe_easy.tar.gz 
bitfinder_awair_breathe_easy.json
Extract candy_house_sesami_wi-fi_access_point.tar.gz 
candy_house_sesami_wi-fi_access_point.json
Extract irobot_roomba.tar.gz 
irobot_roomba.json
Extract jvc_kenwood_cu-hb1.tar.gz 
jvc_kenwood_cu-hb1.json
Extract jvc_kenwood_hdtv_ip_camera.tar.gz 
jvc_kenwood_hdtv_ip_camera.json
Extract line_clova_wave.tar.gz 
line_clova_wave.json
Extract link_japan_eremote.tar.gz 
link_japan_eremote.json
Extract mouse_computer_room_hub.tar.gz 
mouse_computer_room_hub.json
Extract nature_remo.tar.gz 
nature_remo.json
Extract panasonic_doorphone.tar.gz 
panasonic_doorphone.json
Extract philips_hue_bridge.tar.gz 
philips_hue_bridge.json
Extract 'planex_camera_one_shot!.tar.gz' 
planex_camera_one_shot!.json
Extract planex_smacam_outdoor.tar.gz 
planex_smacam

In [5]:
# Print the current working directory
print("Current Working Directory:", os.getcwd())

# List the contents of the current working directory
print("Contents of Current Directory:", os.listdir())

!cd

Current Working Directory: /content/KDDI-IoT-2019/ipfix
Contents of Current Directory: ['sony_smart_speaker.tar.gz', 'jvc_kenwood_cu-hb1.json', 'bitfinder_awair_breathe_easy.json', 'mouse_computer_room_hub.tar.gz', 'jvc_kenwood_hdtv_ip_camera.tar.gz', 'google_home_gen1.tar.gz00', 'au_network_camera.tar.gz', 'sony_network_camera.json', 'xiaomi_mijia_led.json', 'candy_house_sesami_wi-fi_access_point.tar.gz', 'planex_camera_one_shot!.json', 'link_japan_eremote.json', 'link_japan_eremote.tar.gz', 'au_wireless_adapter.json', 'planex_smacam_pantilt.tar.gz', 'sony_network_camera.tar.gz', 'planex_smacam_outdoor.json', 'jvc_kenwood_hdtv_ip_camera.json', 'philips_hue_bridge.json', 'jvc_kenwood_cu-hb1.tar.gz', 'sony_bravia.tar.gz01', 'planex_smacam_outdoor.tar.gz', 'qrio_hub.json', 'philips_hue_bridge.tar.gz', 'amazon_echo_gen2.json', 'candy_house_sesami_wi-fi_access_point.json', 'line_clova_wave.json', 'irobot_roomba.json', 'i-o_data_qwatch.tar.gz01', 'au_network_camera.json', 'line_clova_wave.t

In [77]:
# Code to generate distinct tables for each json file using a limited subset

# Define the directory where the JSON files are located
json_directory = '/content/KDDI-IoT-2019/ipfix'

# Get the list of all JSON files in the directory
json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]

# Create distinct tables for each json file
tables = {}
for json_file in json_files:
    # strip .json suffix from device names
    device_name = json_file.split('.')[0]
    # Construct the full path to the JSON file
    json_path = os.path.join(json_directory, json_file)
    # Read the JSON file into a DataFrame, normalize the 'flows' column, and get the first 1000 rows
    df = pd.json_normalize(pd.read_json(json_path, lines=True, nrows=1000)['flows'])

    # Label the DataFrame with the device name
    df['Device'] = device_name
    tables[device_name] = df

# Concatenate all the Dataframes in the tables dictionary into a single Dataframe
df = pd.concat(tables.values(), ignore_index=True)

#### Discarding certain attributes
The primary goal of training our models is to focus on attributes that provide valuable and distinguishable information about the data

In [78]:
# Define the list of columns to be dropped
drop_columns = ['flowStartMilliseconds',
                'flowEndMilliseconds',
                'sourceMacAddress',
                'destinationMacAddress'
]

# Drop the columns from the dataset
df = df.drop(columns=drop_columns)

In [79]:
import hashlib

# 1. Integer Encoding for IP Addresses
def ip_to_int(ip_str):
    # If IPv4 address
    if '.' in ip_str:
        return int(''.join(ip_str.split('.')))
    # If IPv6 address
    elif ':' in ip_str:
        return int(hashlib.sha256(ip_str.encode('utf-8')).hexdigest(), 16) % 10**8

df['sourceIPv4Address'] = df['sourceIPv4Address'].apply(ip_to_int)
df['destinationIPv4Address'] = df['destinationIPv4Address'].apply(ip_to_int)

# 2. Label Encoding for other categorical attributes
label_encoders = {}
for col in ['flowAttributes', 'initialTCPFlags', 'unionTCPFlags', 'reverseInitialTCPFlags', 'reverseUnionTCPFlags', 'reverseFlowAttributes', 'collectorName', 'flowEndReason', 'firstEightNonEmptyPacketDirections', 'Device']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 3. Convert hex to int
for col in ['tcpSequenceNumber', 'reverseTcpSequenceNumber', 'vlanId', 'ipClassOfService']:
    df[col] = df[col].apply(lambda x: int(x, 16))


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 51 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   flowDurationMilliseconds                  22000 non-null  float64
 1   reverseFlowDeltaMilliseconds              22000 non-null  float64
 2   protocolIdentifier                        22000 non-null  int64  
 3   sourceIPv4Address                         22000 non-null  int64  
 4   sourceTransportPort                       22000 non-null  int64  
 5   packetTotalCount                          22000 non-null  int64  
 6   octetTotalCount                           22000 non-null  int64  
 7   flowAttributes                            22000 non-null  int64  
 8   destinationIPv4Address                    22000 non-null  int64  
 9   destinationTransportPort                  22000 non-null  int64  
 10  reversePacketTotalCount           

In [81]:
# Separate the Device column and flowStartMilliseconds as labels
labels_df = df['Device'].copy()

# Drop the Device column from the original DataFrame
df = df.drop(columns=['Device'])

X = df
y = labels_df

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Dealing with Missing Values in our Dataset

#### Defining function to measure quality of each method

In [82]:
"""
  This function trains a RandomForestRegressor on the given data and then
  predicts on the test data. It computes the MAE (Mean Absolute Error) between
  the predicted values and the true validation target values
"""
def mae_score(X_train, X_test, y_train, y_test):
  rfr_model = RandomForestRegressor(n_estimators=10, random_state=0)
  rfr_model.fit(X_train, y_train)
  pred = rfr_model.predict(X_test)

  # Compute MAE between the predictions and true test value
  return mean_absolute_error(y_test, pred)


#### Method 1: Dropping Columns with Missing Values

In [83]:
# Get columns with missing values
missing_col = [col for col in X_train.columns if X_train[col].isnull().any()]

# Drop the columns in the training and test data
drop_Xtrain = X_train.drop(missing_col, axis=1)
drop_Xtest = X_test.drop(missing_col, axis=1)

# Get the MAE for first approach
print("Method 1 MAE: ")
print(mae_score(drop_Xtrain, drop_Xtest, y_train, y_test))

Method 1 MAE: 
2.47395303030303


#### Method 2: Using imputation techniques to fill in Missing Values

In [84]:
from sklearn.impute import SimpleImputer

# Create impute object
impute = SimpleImputer()

# Apply imputer to data
impute_Xtrain = impute.fit_transform(X_train)
impute_Xtest = impute.transform(X_test)

# Convert numpy arrays back to pandas dataframes
impute_Xtrain = pd.DataFrame(impute_Xtrain, columns=X_train.columns)
impute_Xtest = pd.DataFrame(impute_Xtest, columns=X_test.columns)

# Get the MAE for second approach
print("Method 2 MAE: ")
print(mae_score(impute_Xtrain, impute_Xtest, y_train, y_test))

Method 2 MAE: 
2.4451712121212124


#### Method 3: Extension to Imputation